# 눈 사진 캡쳐
눈 감은 거 1000장
실눈 ~ 정상 ~ 큰눈 1000장

In [ ]:
# 필요한 라이브러리 및 모듈 임포트
import cv2  # OpenCV: 영상 처리 라이브러리
import dlib  # dlib: 얼굴 및 객체 검출 라이브러리
import numpy as np  # NumPy: 수학 및 행렬 연산 라이브러리
from imutils import face_utils  # imutils: OpenCV를 좀 더 편리하게 사용하기 위한 보조 라이브러리
from keras.models import load_model  # Keras: 딥러닝 모델을 쉽게 구축하고 훈련하기 위한 라이브러리

# 상수 정의
IMG_SIZE = (34, 26)  # 눈 이미지 크기

# dlib을 사용하여 얼굴 및 랜드마크 검출을 위한 초기화
detector = dlib.get_frontal_face_detector()  # 얼굴 감지기 초기화
predictor = dlib.shape_predictor('models/shape_predictor_68_face_landmarks.dat')  # 얼굴 랜드마크 예측기 초기화

# 미리 학습된 모델 로드
model = load_model('models/models/2023_12_26_11_40_10.h5')  # 학습된 눈 감지 모델 로드
model.summary()  # 모델 구조 출력

# 눈을 자르고 크기를 조절하는 함수 정의
def crop_eye(img, eye_points):
    # 눈 영역의 경계 좌표 계산
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(int)

    # 이미지에서 눈 영역을 잘라냄
    eye_img = img[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

# 웹캠 열기
capture = cv2.VideoCapture(1)  # 디바이스 아이디
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 2560)  # 웹캠 가로 해상도 설정
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 1440)  # 웹캠 세로 해상도 설정

count = 1590  # 이미지 파일 저장을 위한 카운터 초기화

while cv2.waitKey(10) < 0:  # wait를 늘리면 프레임이 줄어듬
    ret, img_ori = capture.read()
    img = img_ori.copy()
    
    # 그레이스케일로 변환
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 얼굴 및 랜드마크 검출
    faces = detector(gray)
    face = faces[0]
    shapes = predictor(gray, face)
    shapes = face_utils.shape_to_np(shapes)

    # 두 눈을 자르고 크기를 조절
    eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
    eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

    # 눈 이미지 크기 조절 및 좌우 반전
    eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
    eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
    eye_img_r = cv2.flip(eye_img_r, flipCode=1)

    # 이미지 저장 경로 및 파일명 설정
    path_l = f'media/closeEye/{count}.png'
    cv2.imwrite(path_l, eye_img_l)
    count += 1  # 카운터 증가
    path_r = f'media/closeEye/{count}.png'
    cv2.imwrite(path_r, eye_img_r)
    count += 1  # 카운터 증가

    # 모델 입력 형태로 이미지 전처리
    eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
    eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

    # 모델 예측
    pred_l = model.predict(eye_input_l)
    pred_r = model.predict(eye_input_r)

    # 결과 시각화
    state_l = 'Open %.1f' if pred_l > 0.1 else 'Close %.1f'
    state_r = 'Open %.1f' if pred_r > 0.1 else 'Close %.1f'

    state_l = state_l % pred_l
    state_r = state_r % pred_r

    # 결과 시각화를 위한 사각형 및 텍스트 추가
    cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255,255,255), thickness=2)
    cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255,255,255), thickness=2)

    cv2.putText(img, state_l, tuple(eye_rect_l[1:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    cv2.putText(img, state_r, tuple(eye_rect_r[1:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

    # 결과 출력
    cv2.imshow('result', img)

# 웹캠 해제
capture.release()
cv2.destroyAllWindows()


# 눈 데이터셋 만들기

In [ ]:
import os
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 이미지를 읽고 지정된 크기로 조정하는 함수
def read_and_resize(file_path, target_size=(34, 26)):
    img = Image.open(file_path)
    img = img.resize(target_size, Image.ANTIALIAS)
    return np.array(img)

# 각각의 이미지 파일이 저장된 폴더 경로
open_folder_path = ".media/openEye/"
close_folder_path = ".media/closeEye/"

# 각 폴더에 있는 이미지 파일들의 경로를 리스트로 수집
open_image_files = [os.path.join(open_folder_path, f) for f in os.listdir(open_folder_path) if f.endswith(".png")]
close_image_files = [os.path.join(close_folder_path, f) for f in os.listdir(close_folder_path) if f.endswith(".png")]

# 이미지 데이터를 읽어서 리스트에 추가
image_data = []
state = []

# 눈 뜬 사진 추가
for open_file in open_image_files:
    open_data = read_and_resize(open_file)
    image_data.append(open_data)
    state.append("open")

# 눈 감은 사진 추가
for close_file in close_image_files:
    close_data = read_and_resize(close_file)
    image_data.append(close_data)
    state.append("close")

# 데이터셋 생성
data = pd.DataFrame({"state": state, "image": image_data})

# 데이터를 랜덤하게 섞은 후 훈련 데이터와 검증 데이터로 분할
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# CSV 파일로 저장
print("image_data", np.array(image_data))
print("data", data)
data.to_csv("dataset/eye/dataset.csv", index=False)
train_data.to_csv("dataset/eye/train_dataset.csv")
val_data.to_csv("dataset/eye/val_dataset.csv")

# 이미지 데이터를 NumPy 배열로 변환
x_train = np.array(train_data["image"].tolist()).astype(np.float32)
x_val = np.array(val_data["image"].tolist()).astype(np.float32)

# 레이블을 NumPy 배열로 변환
y_train = np.array(train_data["state"].map({"open": 1, "close": 0})).reshape(-1, 1).astype(np.float32)
y_val = np.array(val_data["state"].map({"open": 1, "close": 0})).reshape(-1, 1).astype(np.float32)

# NumPy 배열을 파일로 저장
np.save("dataset/eye/x_train.npy", x_train)
np.save("dataset/eye/x_val.npy", x_val)
np.save("dataset/eye/y_train.npy", y_train)
np.save("dataset/eye/y_val.npy", y_val)

# 출력 확인
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)


# 입술 따기
입 다문 거 500장, 말하는 거 500장  
하품하듯이 입 크게 벌린 거 1000장

In [ ]:
import cv2
import dlib
import os

# 얼굴 랜드마크 모델 로드
predictor_path = "models/shape_predictor_68_face_landmarks.dat"  # 모델 경로를 실제 파일 경로로 변경해주세요
predictor = dlib.shape_predictor(predictor_path)

# 얼굴 감지기 로드
detector = dlib.get_frontal_face_detector()

# 웹캠 열기
cap = cv2.VideoCapture(1)

# 웹캠 해상도 설정 (720p)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # 가로 해상도
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)   # 세로 해상도

# 웹캠 프레임 속도 설정 (60fps)
cap.set(cv2.CAP_PROP_FPS, 60)

# 크롭할 이미지 크기
crop_size = 100

# ??번째부터
current_frame = 1200

# +100번째까지
num_frames_to_capture = current_frame + 100



while current_frame < num_frames_to_capture:
    ret, frame = cap.read()

    # BGR 이미지를 그레이스케일로 변환
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 감지
    faces = detector(gray_frame)

    # 얼굴이 감지되었다면
    if faces:
        # 가장 큰 얼굴 1개 찾기
        largest_face = max(faces, key=lambda rect: (rect.width() * rect.height()))

        # 얼굴 랜드마크 예측
        landmarks = predictor(gray_frame, largest_face)

        # 입술 중심 좌표 계산
        lip_center_x = sum(point.x for point in landmarks.parts()[48:60]) // 12
        lip_center_y = sum(point.y for point in landmarks.parts()[48:60]) // 12

        # 입 중심을 기준으로 10 x 10 크롭
        lip_min_x = max(0, lip_center_x - crop_size // 2)
        lip_max_x = min(gray_frame.shape[1], lip_center_x + crop_size // 2)
        lip_min_y = max(0, lip_center_y - crop_size // 2)
        lip_max_y = min(gray_frame.shape[0], lip_center_y + crop_size // 2)

        lip_region = gray_frame[lip_min_y:lip_max_y, lip_min_x:lip_max_x]

        # 이미지 저장
        save_path = f"media/openLip/{current_frame}.png"  # 저장
        cv2.imwrite(save_path, lip_region)
        print(f"Frame {current_frame} saved at {save_path}")

        current_frame += 1

    # 화면에 표시
    cv2.imshow("Lip Landmark Detection", gray_frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 웹캠 해제 및 창 닫기
cap.release()
cv2.destroyAllWindows()


# 입 데이터셋

In [1]:
import os
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 이미지를 읽고 지정된 크기로 조정하는 함수
def read_and_resize(file_path, target_size=(10, 10)):
    img = Image.open(file_path)
    img = img.resize(target_size, Image.ANTIALIAS)
    return np.array(img)

# 각각의 이미지 파일이 저장된 폴더 경로
open_folder_path = ".media/openLip/"
close_folder_path = ".media/closeLip/"

# 각 폴더에 있는 이미지 파일들의 경로를 리스트로 수집
open_image_files = [os.path.join(open_folder_path, f) for f in os.listdir(open_folder_path) if f.endswith(".png")]
close_image_files = [os.path.join(close_folder_path, f) for f in os.listdir(close_folder_path) if f.endswith(".png")]

# 이미지 데이터를 읽어서 리스트에 추가
image_data = []
state = []

# 하품 사진 추가
for open_file in open_image_files:
    open_data = read_and_resize(open_file)
    image_data.append(open_data)
    state.append("open")

# 입 닫거나 말하는 사진 추가
for close_file in close_image_files:
    close_data = read_and_resize(close_file)
    image_data.append(close_data)
    state.append("close")

# 데이터셋 생성
data = pd.DataFrame({"state": state, "image": image_data})

# 데이터를 랜덤하게 섞은 후 훈련 데이터와 검증 데이터로 분할
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# CSV 파일로 저장
print("image_data", np.array(image_data))
print("data", data)
data.to_csv("dataset/lip/dataset.csv", index=False)
train_data.to_csv("dataset/lip/train_dataset.csv")
val_data.to_csv("dataset/lip/val_dataset.csv")

# 이미지 데이터를 NumPy 배열로 변환
x_train = np.array(train_data["image"].tolist()).astype(np.float32)
x_val = np.array(val_data["image"].tolist()).astype(np.float32)

# 레이블을 NumPy 배열로 변환
y_train = np.array(train_data["state"].map({"open": 1, "close": 0})).reshape(-1, 1).astype(np.float32)
y_val = np.array(val_data["state"].map({"open": 1, "close": 0})).reshape(-1, 1).astype(np.float32)

# NumPy 배열을 파일로 저장
np.save("dataset/lip/x_train.npy", x_train)
np.save("dataset/lip/x_val.npy", x_val)
np.save("dataset/lip/y_train.npy", y_train)
np.save("dataset/lip/y_val.npy", y_val)

# 출력 확인
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)


/var/folders/wl/5h8ygnjj29ndbpccd4zxqbfh0000gn/T/ipykernel_59133/1696903667.py:10: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(target_size, Image.ANTIALIAS)


image_data [[[122 145 152 ... 153 154 146]
  [117  94  94 ... 125 147 149]
  [ 51   9  18 ...  46  53 100]
  ...
  [115  93  90 ... 102 117 128]
  [123 103  95 ... 112 131 128]
  [121 121 101 ... 125 131 126]]

 [[161 185 181 ... 196 194 188]
  [173 165 143 ... 158 181 183]
  [172 141 117 ...  88 117 163]
  ...
  [177 181 176 ... 160 179 170]
  [170 179 166 ... 170 161 178]
  [160 175 167 ... 150 173 181]]

 [[118 143 146 ... 159 171 163]
  [113  84  96 ... 113 124 147]
  [ 51   6  24 ...  42  26  59]
  ...
  [116  93  92 ...  86 102 130]
  [121 101  99 ... 133 121 133]
  [119 117 102 ... 123 128 133]]

 ...

 [[158 151 128 ... 166 178 189]
  [160 156 164 ... 189 180 185]
  [153 163 160 ... 163 192 183]
  ...
  [125 112 102 ... 103 116 131]
  [125 120 111 ... 125 131 134]
  [136 137 131 ... 152 154 151]]

 [[134 145 133 ... 156 158 157]
  [142 145 151 ... 171 163 165]
  [143 154 160 ... 183 178 165]
  ...
  [129 125 117 ... 118 128 135]
  [135 130 119 ... 126 139 146]
  [143 144 134 ..